# Landslide Hazard Analysis Using SAR
Intro to the use case/methods used

Search for SLC images over Haiti in a defined time period

In [ ]:
from datetime import datetime

start_time = datetime.strptime('2017-08-01T23:59', '%Y-%m-%dT%H:%M')
end_time = datetime.strptime('2021-08-16T23:59', '%Y-%m-%dT%H:%M')
event_time = datetime.strptime('2021-09-14T00:00', '%Y-%m-%dT%H:%M')

import asf_search
scenes_to_submit = []
wkt = 'POLYGON((-74.6 18.0,-73.0 18.0,-73.0 18.8,-74.6 18.8, -74.6 18.0))'
results = asf_search.geo_search(platform=[asf_search.PLATFORM.SENTINEL1], intersectsWith=wkt, processingLevel='SLC', start=start_time, end=end_time)
[scenes_to_submit.append(result.properties['sceneName']) for result in results]
print(f'There are {len(scenes_to_submit)} scenes in the AOI between {start_time} and {end_time}')

Request HyP3 processing of Haiti data for two years

In [ ]:
import hyp3_sdk as sdk

print(f'Submitting {len(scenes_to_submit)} jobs')
hyp3 = sdk.HyP3()
rtc_jobs = sdk.Batch()
for scene in scenes_to_submit:
    rtc_jobs += hyp3.submit_rtc_job(granule=scene, name='IGARSS-HAITI')

In [ ]:
rtc_jobs = hyp3.watch(rtc_jobs)
succeeded_jobs = rtc_jobs.filter_jobs(succeeded=True, running=False, failed=False)
file_list = succeeded_jobs.download_files()

Download the staged version of the data

In [ ]:
import boto3
import os
import zipfile
from pathlib import Path
from botocore import UNSIGNED
from botocore.client import Config

s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))

for file in file_list:
    download_path = Path(file)
    s3_client.download_file('jrsmale-shenanigans', f'igarss_2023/{file}', download_path)
    with zipfile.ZipFile(download_path, 'r') as zip_ref:
        zip_ref.extractall(download_path.stem)
    os.unlink(download_path)

Crop all images to same extent (e.g. InSAR Notebook)

In [ ]:
from pathlib import Path
from typing import List, Union
from osgeo import gdal

def clip_hyp3_products_to_same_extent(data_dir: Union[str, Path], extent: List[float]) -> List[str]:
    """Clip all GeoTIFF hyp3 products to a specified extent
    Args:
        data_dir:
            directory containing the GeoTIFF files to clip
        extent:
            a list of the upper-left x, upper-left y, lower-right x, and lower-right y corner coordinates of desired extent.
    Returns: None
    """
    files_for_xarray = ['VV.tif', 'VH.tif', '_ls_map.tif']

    for extension in files_for_xarray:
        print(extension)
        for file in data_dir.glob(f'**/*{extension}'):
            dst_file = Path(str(file.absolute()).replace(f'{extension}', f'{extension[:-4]}_clipped.tif'))
            if not Path(dst_file).is_file():
                gdal.Translate(destName=str(dst_file), srcDS=str(file), projWin=extent)#, projWinSRS="EPSG:4326")

In [ ]:
data_dir = Path('/Users/jrsmale/data/unzipped_IGARSS/')
extent = [544823, 2071376, 683411, 1987136]
clip_hyp3_products_to_same_extent(data_dir, extent)

Load the geotiffs into Xarray with datetime stamps for each image

In [ ]:
def extract_tif_names(scene_dir_path):
    scene_path_listed = os.listdir(scene_dir_path)
    scene_files_vv = [fname for fname in scene_path_listed if fname.endswith('_VV_clipped.tif')]
    scene_files_vh = [fname for fname in scene_path_listed if fname.endswith('_VH_clipped.tif')]
    scene_files_ls = [fname for fname in scene_path_listed if fname.endswith('_ls_map_clipped.tif')]
    scene_file_rm = [fname for fname in scene_path_listed if fname.endswith('README.md.txt')]
    return scene_files_vv, scene_files_vh, scene_files_ls, scene_file_rm

scenes_listed = os.listdir(data_dir)
fpaths_vv, fpaths_vh, fpaths_ls, fpaths_rm = [],[],[], []

for element in range(len(scenes_listed)):
    print(f'{data_dir}/{scenes_listed[element]}')
    try:
        good_files = extract_tif_names(f'{data_dir}/{scenes_listed[element]}')
        path_vh = f'{data_dir}/{scenes_listed[element]}/{good_files[0][0]}'
        path_vv = f'{data_dir}/{scenes_listed[element]}/{good_files[1][0]}'
        path_ls = f'{data_dir}/{scenes_listed[element]}/{good_files[2][0]}'
        path_readme = f'{data_dir}/{scenes_listed[element]}/{good_files[3][0]}'

        fpaths_vv.append(path_vv)
        fpaths_vh.append(path_vh)
        fpaths_ls.append(path_ls)
        fpaths_rm.append(path_readme)
    except:
        print(f'Couldnt use {scenes_listed[element]}. Skipping.')
        continue

gdal.BuildVRT(str(data_dir)+'/VV.vrt', fpaths_vv, separate=True)
gdal.BuildVRT(str(data_dir)+'/VH.vrt', fpaths_vh, separate=True)
gdal.BuildVRT(str(data_dir)+'/ls.vrt', fpaths_ls, separate=True)

Pre-temporal averaging, create time series plot of single pixel (or group of pixels)

In [ ]:
import rioxarray as rio
import xarray as xr
vrt_vv = rio.open_rasterio(str(data_dir)+'/VV.vrt', chunks='auto').squeeze()
vrt_vh = rio.open_rasterio(str(data_dir)+'/VH.vrt', chunks='auto').squeeze()
vrt_ls = rio.open_rasterio(str(data_dir)+'/ls.vrt', chunks='auto').squeeze()
import pandas as pd

def extract_metadata_attrs(inputfname):
    print(inputfname)
    sensor = inputfname[0:3]
    beam_mode = inputfname[4:6]
    acq_date_full = inputfname[7:22]
    acq_datetime = pd.to_datetime(acq_date_full, format='%Y%m%dT%H%M%S')
    pol_type = inputfname[23:25]
    orbit_type = inputfname[25:26]
    orbit_type = inputfname[26:27] #Precise (P), Restituted (R), or Original Predicted (O)
    terrain_correction_pixel_spacing = inputfname[27:32] #Terrain Correction Pixel Spacing
    rtc_alg = inputfname[33:34] #Software Package Used: GAMMA (G)
    output = inputfname[35] #  Gamma-0 (g) or Sigma-0 (s) Output
    output_type = inputfname[36] #Power (p) or Decibel (d) or Amplitude (a) Output
    masked = inputfname[37]  #Unmasked (u) or Water Masked (w)
    filtered = inputfname[38]  # Not Filtered (n) or Filtered (f)
    area =  inputfname[39]       # Entire Area (e) or Clipped Area (c)
    tbd =   inputfname[40]   #Dead Reckoning (d) or DEM Matching (m)
   # product_id  = inputfname[56:62]  #Product ID
    product_id = inputfname[42:46]

    attrs_dict = { 'sensor': sensor,
                        'beam_mode':beam_mode,
                        'acquisition_datetime': acq_datetime,
                        'polarisation_type': pol_type,
                     #  'primary_polarisation': primary_pol,
                        'orbit_type': orbit_type,
                        'terrain_correction_pixel_spacing' : terrain_correction_pixel_spacing,
                        'output_format': output,
                        'output_type': output_type,
                        'masked' : masked,
                        'filtered':filtered,
                        'area':area,
                        'product_id': product_id
                 }
    return attrs_dict

acq_dates_vh = [extract_metadata_attrs(Path(file).name)['acquisition_datetime'].strftime('%m/%d/%YT%H%M%S') for file in
                fpaths_vh]
acq_dates_vv = [extract_metadata_attrs(Path(file).name)['acquisition_datetime'].strftime('%m/%d/%YT%H%M%S') for file in
                fpaths_vv]
acq_dates_ls = [extract_metadata_attrs(Path(file).name)['acquisition_datetime'].strftime('%m/%d/%YT%H%M%S') for file in
                fpaths_ls]

vrt_vv = vrt_vv.assign_coords({'acq_date':pd.to_datetime(acq_dates_vv, format='%m/%d/%YT%H%M%S')})
vrt_vh = vrt_vh.assign_coords({'band':pd.to_datetime(acq_dates_vh, format='%m/%d/%YT%H%M%S')})
vrt_ls = vrt_ls.assign_coords({'band':pd.to_datetime(acq_dates_ls, format='%m/%d/%YT%H%M%S')})

vrt_merged = xr.Dataset({'vv': vrt_vv,
                        'vh': vrt_vh,
                        'ls': vrt_ls}).rename_dims({'band': 'acq_date'})

vrt_merged = vrt_merged.sortby(vrt_merged.acq_date)

meta_attrs_ls = [extract_metadata_attrs(Path(file).name) for file in fpaths_ls]
meta_attrs_vv = [extract_metadata_attrs(Path(file).name) for file in fpaths_vv]
meta_attrs_vh = [extract_metadata_attrs(Path(file).name) for file in fpaths_vh]

import markdown
def stacked_meta_tuple(input_ls):
    '''takes a list of dictionaries where each dict is metadata for a given time step,
    returns a tuple. input list is re-organized to a tuple where each [0] values is a metadata category (ie. sensor) and
    [1] is a time series of the categories values over time (ie. S1A, S1A, S1A ...)'''

    meta_dict = input_ls[0]
    ticker = 0
    attrs_dicts, keys_ls = [],[]

    for key in meta_dict:
        if key == 'acquisition_datetime':
            pass
        else:

            key_dict = {f'{key}':[input_ls[ticker][key] for ticker in range(len(input_ls))]}
            ticker+=1
            attrs_dicts.append(key_dict)
            keys_ls.append(key)

    full_tuple = tuple(zip(keys_ls, attrs_dicts))

    return full_tuple

def apply_meta_coords(input_xr, input_tuple):
    '''takes an input xarray object and a tuple of metadata created from the above fn.
    returns xr object with metadata from tuple applied
    '''

    out_xr = xr.Dataset(
        input_xr.data_vars,
        coords = {'x': input_xr.x.data,
                  'y': input_xr.y.data,
        #          'acq_date': input_xr.acq_date.data,
                 }
    )
    # now apply metadata coords
    for element in range(len(input_tuple)):
        key = input_tuple[element][0]
        coord = list(input_tuple[element][1].values())[0]
        out_xr.coords[f'{key}'] = ('acq_date', coord)
       # print(key)
       # print(out_xr.coords.values)

    return out_xr

import numpy as np
meta_tuple = stacked_meta_tuple(meta_attrs_vv)
vrt_full = apply_meta_coords(vrt_merged, meta_tuple)


def extract_granule_id(filepath):
    ''' takes a filepath to the readme associated with an S1 scene and returns the source granule id used to generate the RTC imagery'''
    data = Path(filepath).read_text()
    #this text precedes granule ID in readme
    gran_str = 'The source granule used to generate the products contained in this folder is:\n'
    split = data.split(gran_str)
    #isolate the granule id
    gran_id = split[1][:67]

    return gran_id

def make_granule_coord(readme_fpaths_ls):
    '''takes a list of the filepaths to every read me, extracts the granule ID,
    extracts acq date for each granule ID, organizes this as an array that
    can be assigned as a coord to an xarray object'''

    granule_ls = [extract_granule_id(readme_fpaths_ls[element]) for element in range(len(readme_fpaths_ls))]
    acq_date = [pd.to_datetime(granule[17:25]) for granule in granule_ls]
    granule_da = xr.DataArray(data = granule_ls,
                              dims = ['acq_date'],
                              coords = {'acq_date':acq_date},
                              attrs = {'description': 'source granule ID for ASF-processed S1 RTC imagery, extracted from README files for each scene'},
                              name = 'granule_id')
    granule_da = granule_da.sortby(granule_da.acq_date)

    return granule_da
granule_da = make_granule_coord(fpaths_rm)
vrt_full.coords["granule_id"] = ('acq_date', granule_da.data)
vrt_full =  vrt_full.where(vrt_full.vv != 0., np.nan, drop=False)
vrt_full.to_netcdf(str(data_dir)+'/prepared_Haiti_data.nc')

Create a temporal average for both pre and post EQ

In [ ]:
pre_event_average = xr.slice

Perform a log difference of the two resulting images.

Discuss options for thresholding the difference image

Use the COP30 DEM to remove change detections in areas with slopes <5 degrees use gdaldem

 Plot a histogram for the difference image (might have to subset histogram source to area with equal amounts of landslides and non-landslides).

Create a figure to interactively view the data and set a threshold using matplotlib